## Super resolution

In [ ]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *

from torchvision.models import vgg16_bn

from ipyexperiments import *
exp1 = IPyExperimentsPytorch()
import objgraph



# w/o fixing random memory usage patterns change wildly and it's hard to tune things up this way
if True:
    import torch
    torch.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed_all(42)
    random.seed(42)
    np.random.seed(42)



*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)


*** Current state:
RAM:  Used  Free  Total      Util
CPU:  1901 20119  31588 MB   6.02% 
GPU:   427  7692   8119 MB   5.26% 




･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.002
･ CPU:         0       0     1901 MB |
･ GPU:         0       0      427 MB |


In [ ]:
path = untar_data(URLs.PETS)
path_hr = path/'images'
path_lr = path/'small-96'
path_mr = path/'small-256'

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.003
･ CPU:         0       1     1901 MB |
･ GPU:         0       0      427 MB |


In [ ]:
il = ImageItemList.from_folder(path_hr)

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.064
･ CPU:         2       0     1901 MB |
･ GPU:         0       0      427 MB |


In [ ]:
def resize_one(fn, i, path, size):
    dest = path/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn)
    targ_sz = resize_to(img, size, use_min=True)
    img = img.resize(targ_sz, resample=PIL.Image.BILINEAR).convert('RGB')
    img.save(dest, quality=60)

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.001
･ CPU:         0       0     1901 MB |
･ GPU:         0       0      427 MB |


In [ ]:
# create smaller image sets the first time this nb is run
sets = [(path_lr, 96), (path_mr, 256)]
for p,size in sets:
    if not p.exists(): 
        print(f"resizing to {size} into {p}")
        parallel(partial(resize_one, path=p, size=size), il.items)

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.037
･ CPU:         0       0     1901 MB |
･ GPU:         0       0      427 MB |


In [ ]:
bs,size=32,128
arch = models.resnet34

src = ImageImageList.from_folder(path_lr).random_split_by_pct(0.1, seed=42)

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.099
･ CPU:         1       0     1902 MB |
･ GPU:         0       0      427 MB |


In [ ]:
def get_data(bs,size):
    data = (src.label_from_func(lambda x: path_hr/x.name)
           .transform(get_transforms(max_zoom=2.), size=size, tfm_y=True)
           .databunch(bs=bs).normalize(imagenet_stats, do_y=True))

    data.c = 3
    return data

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.001
･ CPU:         0       0     1902 MB |
･ GPU:         0       0      427 MB |


In [ ]:
data = get_data(bs,size)

･ RAM: △Consumed △Peaked  Used Total | Exec time 0:00:00.352
･ CPU:         1       3     1904 MB |
･ GPU:         0       0      427 MB |


In [ ]:
#data.show_batch(ds_type=DatasetType.Valid, rows=2, figsize=(9,9))

## Feature loss

In [ ]:
t = data.valid_ds[0][1].data
t = torch.stack([t,t])

def gram_matrix(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

gram_matrix(t)

base_loss = F.l1_loss

vgg_m = vgg16_bn(True).features.cuda().eval()
requires_grad(vgg_m, False)

blocks = [i-1 for i,o in enumerate(children(vgg_m)) if isinstance(o,nn.MaxPool2d)]
blocks, [vgg_m[i] for i in blocks]

class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        self.hooks = hook_outputs(self.loss_features, detach=False)
        self.wgts = layer_wgts
        self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layer_ids))
              ] + [f'gram_{i}' for i in range(len(layer_ids))]

    def make_features(self, x, clone=False):
        self.m_feat(x)
        return [(o.clone() if clone else o) for o in self.hooks.stored]
    
    def forward(self, input, target):
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [base_loss(input,target)]
        self.feat_losses += [base_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.metrics = dict(zip(self.metric_names, self.feat_losses))
        return sum(self.feat_losses)
    
    def __del__(self): print("hook remove!!!"); self.hooks.remove()

In [ ]:
feat_loss = FeatureLoss(vgg_m, blocks[2:5], [5,15,2])

## Train

In [ ]:
wd = 1e-3
#loss_func=feat_loss
#loss_func=F.l1_loss 
learn = unet_learner(data, arch, wd=wd, 
                     #loss_func=F.l1_loss,
                     loss_func=feat_loss, callback_fns=LossMetrics,
                     blur=True, norm_type=NormType.Weight)
del feat_loss # very important memory saving down the road

In [ ]:
#_=learn.summary()

In [ ]:
#gc.collect()
#sys.getrefcount(learn.loss_func)
#sys.getrefcount(feat_loss)
#len(gc.get_referrers(learn.loss_func))
#del learn.loss_func
#for x in gc.get_referrers(feat_loss):
#    print(x.keys())
    

#del feat_loss
#len(gc.get_referrers(learn.loss_func))
#len(gc.get_referrers(learn.loss_func))
#len(gc.get_referrers(feat_loss))
#feat_loss.hooks.remove()
#gc.get_referrers(feat_loss)

In [ ]:
#objgraph.show_refs(feat_loss)

In [ ]:
#feat_loss.hooks.remove()
#objgraph.show_refs(feat_loss)

In [ ]:
_=learn.summary()

In [ ]:
#learn.purge()
#learn.destroy()

In [ ]:
#die


In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
#del learn.loss_func
#del feat_loss
#del loss_func
#learn.destroy()

In [ ]:
_=learn.purge(clear_opt=False)

In [ ]:
#del vgg_m
#del learn
#gc.collect()
#torch.cuda.empty_cache()

In [ ]:
lr = 1e-3

In [ ]:
def do_fit(save_name, lrs=slice(lr), pct_start=0.9):
    # XXX:
    cycles = 1
    learn.fit_one_cycle(cycles, lrs, pct_start=pct_start)
    learn.save(save_name)
    learn.show_results(rows=1, imgsize=5)

In [ ]:
do_fit('1a', slice(lr*10))

In [ ]:
1

In [ ]:
learn.destroy()

In [ ]:
1

In [ ]:
die

In [ ]:
learn.unfreeze()

In [ ]:
do_fit('1b', slice(1e-5,lr))

In [ ]:
data = get_data(12,size*2)

In [ ]:
learn.data = data
learn.freeze()
gc.collect()

In [ ]:
learn.load('1b');

In [ ]:
do_fit('2a')

In [ ]:
learn.unfreeze()

In [ ]:
do_fit('2b', slice(1e-6,1e-4), pct_start=0.3)

## Test

In [ ]:
learn = None
gc.collect();

In [ ]:
256/320*1024

In [ ]:
256/320*1600

In [ ]:
free = gpu_mem_get_free_no_cache()
# the max size of the test image depends on the available GPU RAM 
if free > 8000: size=(1280, 1600) # >  8GB RAM
else:           size=( 820, 1024) # <= 8GB RAM
print(f"using size={size}, have {free}MB of GPU RAM free")

In [ ]:
learn = unet_learner(data, arch, loss_func=F.l1_loss, blur=True, norm_type=NormType.Weight)

In [ ]:
data_mr = (ImageImageList.from_folder(path_mr).random_split_by_pct(0.1, seed=42)
          .label_from_func(lambda x: path_hr/x.name)
          .transform(get_transforms(), size=size, tfm_y=True)
          .databunch(bs=1).normalize(imagenet_stats, do_y=True))
data_mr.c = 3

In [ ]:
learn.load('2b');

In [ ]:
learn.data = data_mr

In [ ]:
fn = data_mr.valid_ds.x.items[0]; fn

In [ ]:
img = open_image(fn); img.shape

In [ ]:
p,img_hr,b = learn.predict(img)

In [ ]:
show_image(img, figsize=(18,15), interpolation='nearest');

In [ ]:
Image(img_hr).show(figsize=(18,15))